## Introducción a la Regresión - Lección 1

#### Poniéndolo en perspectiva

✅ Hay muchos tipos de métodos de regresión, y cuál elijas depende de la respuesta que estás buscando. Si quieres predecir la altura probable de una persona dada su edad, usarías `regresión lineal`, ya que estás buscando un **valor numérico**. Si estás interesado en descubrir si un tipo de cocina debería considerarse vegana o no, estás buscando una **asignación de categoría**, por lo que usarías `regresión logística`. Aprenderás más sobre la regresión logística más adelante. Piensa un poco en algunas preguntas que podrías hacerle a los datos y cuál de estos métodos sería más apropiado.

En esta sección, trabajarás con un [pequeño conjunto de datos sobre diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Imagina que quisieras probar un tratamiento para pacientes diabéticos. Los modelos de aprendizaje automático podrían ayudarte a determinar qué pacientes responderían mejor al tratamiento, basándote en combinaciones de variables. Incluso un modelo de regresión muy básico, cuando se visualiza, podría mostrar información sobre variables que te ayudarían a organizar tus ensayos clínicos teóricos.

Dicho esto, ¡vamos a comenzar con esta tarea!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Obra de @allison_horst</figcaption>

<!--![Obra de \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.es.jpg)<br>Obra de @allison_horst-->


## 1. Cargando nuestro conjunto de herramientas

Para esta tarea, necesitaremos los siguientes paquetes:

-   `tidyverse`: El [tidyverse](https://www.tidyverse.org/) es una [colección de paquetes de R](https://www.tidyverse.org/packages) diseñada para hacer la ciencia de datos más rápida, fácil y divertida.

-   `tidymodels`: El marco de trabajo [tidymodels](https://www.tidymodels.org/) es una [colección de paquetes](https://www.tidymodels.org/packages/) para modelado y aprendizaje automático.

Puedes instalarlos con el siguiente comando:

`install.packages(c("tidyverse", "tidymodels"))`

El script a continuación verifica si tienes los paquetes necesarios para completar este módulo y los instala por ti en caso de que falte alguno.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Ahora, carguemos estos paquetes increíbles y hagámoslos disponibles en nuestra sesión actual de R. (Esto es solo para ilustración, `pacman::p_load()` ya lo hizo por ti)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. El conjunto de datos de diabetes

En este ejercicio, pondremos en práctica nuestras habilidades de regresión haciendo predicciones sobre un conjunto de datos de diabetes. El [conjunto de datos de diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) incluye `442 muestras` de datos relacionados con la diabetes, con 10 variables predictoras: `edad`, `sexo`, `índice de masa corporal`, `presión arterial promedio` y `seis mediciones de suero sanguíneo`, así como una variable de resultado `y`: una medida cuantitativa de la progresión de la enfermedad un año después del punto de referencia.

|Número de observaciones|442|
|-----------------------|:---|
|Número de predictores|Las primeras 10 columnas son numéricas predictivas|
|Resultado/Objetivo|La columna 11 es una medida cuantitativa de la progresión de la enfermedad un año después del punto de referencia|
|Información de los predictores|- edad en años
||- sexo
||- bmi índice de masa corporal
||- bp presión arterial promedio
||- s1 tc, colesterol total en suero
||- s2 ldl, lipoproteínas de baja densidad
||- s3 hdl, lipoproteínas de alta densidad
||- s4 tch, colesterol total / HDL
||- s5 ltg, posiblemente logaritmo del nivel de triglicéridos en suero
||- s6 glu, nivel de azúcar en sangre|

> 🎓 Recuerda, esto es aprendizaje supervisado, y necesitamos un objetivo llamado 'y'.

Antes de poder manipular datos con R, necesitas importar los datos a la memoria de R o establecer una conexión con los datos para que R pueda acceder a ellos de forma remota.

> El paquete [readr](https://readr.tidyverse.org/), que forma parte de Tidyverse, proporciona una forma rápida y amigable de leer datos rectangulares en R.

Ahora, carguemos el conjunto de datos de diabetes proporcionado en esta URL fuente: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Además, realizaremos una verificación básica de nuestros datos usando `glimpse()` y mostraremos las primeras 5 filas usando `slice()`.

Antes de continuar, también introduzcamos algo que encontrarás con frecuencia en el código de R 🥁🥁: el operador pipe `%>%`

El operador pipe (`%>%`) realiza operaciones en secuencia lógica pasando un objeto hacia adelante dentro de una función o expresión de llamada. Puedes pensar en el operador pipe como si dijera "y luego" en tu código.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` nos muestra que estos datos tienen 442 filas y 11 columnas, y todas las columnas son del tipo de dato `double`.

<br>

> glimpse() y slice() son funciones de [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, que forma parte de Tidyverse, es una gramática para la manipulación de datos que proporciona un conjunto consistente de verbos que te ayudan a resolver los desafíos más comunes en la manipulación de datos.

<br>

Ahora que tenemos los datos, vamos a centrarnos en una característica (`bmi`) como objetivo para este ejercicio. Esto requerirá que seleccionemos las columnas deseadas. Entonces, ¿cómo hacemos esto?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) nos permite *seleccionar* (y opcionalmente renombrar) columnas en un marco de datos.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Datos de entrenamiento y prueba

Es una práctica común en el aprendizaje supervisado *dividir* los datos en dos subconjuntos: un conjunto (generalmente más grande) para entrenar el modelo, y un conjunto más pequeño de "reserva" para evaluar cómo se desempeñó el modelo.

Ahora que tenemos los datos listos, podemos ver si una máquina puede ayudar a determinar una división lógica entre los números de este conjunto de datos. Podemos usar el paquete [rsample](https://tidymodels.github.io/rsample/), que forma parte del marco Tidymodels, para crear un objeto que contenga la información sobre *cómo* dividir los datos, y luego dos funciones adicionales de rsample para extraer los conjuntos de entrenamiento y prueba creados:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Entrenar un modelo de regresión lineal con Tidymodels

¡Ahora estamos listos para entrenar nuestro modelo!

En Tidymodels, se especifican los modelos utilizando `parsnip()` definiendo tres conceptos:

-   El **tipo** de modelo diferencia entre modelos como la regresión lineal, la regresión logística, los modelos de árboles de decisión, entre otros.

-   El **modo** del modelo incluye opciones comunes como regresión y clasificación; algunos tipos de modelos admiten cualquiera de estos modos, mientras que otros solo tienen un modo disponible.

-   El **motor** del modelo es la herramienta computacional que se utilizará para ajustar el modelo. A menudo, estos son paquetes de R, como **`"lm"`** o **`"ranger"`**.

Esta información del modelo se captura en una especificación de modelo, ¡así que vamos a construir una!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Después de que un modelo ha sido *especificado*, el modelo puede ser `estimado` o `entrenado` utilizando la función [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), generalmente usando una fórmula y algunos datos.

`y ~ .` significa que ajustaremos `y` como la cantidad/objetivo predicho, explicado por todos los predictores/características, es decir, `.` (en este caso, solo tenemos un predictor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

A partir del resultado del modelo, podemos observar los coeficientes aprendidos durante el entrenamiento. Representan los coeficientes de la línea de mejor ajuste que nos proporciona el menor error global entre la variable real y la predicha.
<br>

## 5. Realizar predicciones en el conjunto de prueba

Ahora que hemos entrenado un modelo, podemos usarlo para predecir la progresión de la enfermedad y para el conjunto de datos de prueba utilizando [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Esto se utilizará para trazar la línea entre los grupos de datos.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

¡Qué emoción! 💃🕺 ¡Acabamos de entrenar un modelo y lo usamos para hacer predicciones!

Al hacer predicciones, la convención de tidymodels es siempre generar un tibble/data frame de resultados con nombres de columnas estandarizados. Esto facilita combinar los datos originales con las predicciones en un formato útil para operaciones posteriores, como la creación de gráficos.

`dplyr::bind_cols()` une de manera eficiente múltiples data frames por columna.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Graficar los resultados del modelo

Ahora es momento de verlo de forma visual 📈. Crearemos un diagrama de dispersión con todos los valores de `y` y `bmi` del conjunto de prueba, y luego utilizaremos las predicciones para dibujar una línea en el lugar más adecuado, entre los agrupamientos de datos del modelo.

R tiene varios sistemas para crear gráficos, pero `ggplot2` es uno de los más elegantes y versátiles. Esto te permite componer gráficos **combinando componentes independientes**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Piensa un poco en lo que está sucediendo aquí. Una línea recta atraviesa muchos pequeños puntos de datos, pero ¿qué está haciendo exactamente? ¿Puedes ver cómo deberías poder usar esta línea para predecir dónde debería encajar un nuevo punto de datos no visto en relación con el eje y del gráfico? Intenta expresar en palabras el uso práctico de este modelo.

¡Felicidades, construiste tu primer modelo de regresión lineal, creaste una predicción con él y la mostraste en un gráfico!



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
